In [ ]:
# Lets get the country-list of the CIA WFB!

In [1]:
import requests
import lxml.html as lx

In [2]:
url='https://www.cia.gov/the-world-factbook/countries/'
r=requests.get(url)
html=lx.fromstring(r.text)

In [7]:
r.text.find("Afghanistan") # some countries are present

234195

In [8]:
r.text.find("Germany") # others not

-1

In [13]:
html.xpath('//h2/a/@href') # (incomplete) linklist, find all h2 element; select child a; href attribute

['/the-world-factbook/countries/afghanistan/',
 '/the-world-factbook/countries/akrotiri-and-dhekelia/',
 '/the-world-factbook/countries/albania/',
 '/the-world-factbook/countries/algeria/',
 '/the-world-factbook/countries/american-samoa/',
 '/the-world-factbook/countries/andorra/',
 '/the-world-factbook/countries/angola/',
 '/the-world-factbook/countries/anguilla/',
 '/the-world-factbook/countries/antarctica/',
 '/the-world-factbook/countries/antigua-and-barbuda/',
 '/the-world-factbook/countries/argentina/',
 '/the-world-factbook/countries/armenia/']

In [9]:
# set up the webdriver as in the lecture 

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.chrome.options import Options

In [10]:
chrome_options = Options()

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()), 
    options = chrome_options)

In [11]:
driver.get(url) # 

In [12]:
html=lx.fromstring(driver.page_source)
html.xpath('//h2/a/@href') 

['/the-world-factbook/countries/afghanistan/',
 '/the-world-factbook/countries/akrotiri-and-dhekelia/',
 '/the-world-factbook/countries/albania/',
 '/the-world-factbook/countries/algeria/',
 '/the-world-factbook/countries/american-samoa/',
 '/the-world-factbook/countries/andorra/',
 '/the-world-factbook/countries/angola/',
 '/the-world-factbook/countries/anguilla/',
 '/the-world-factbook/countries/antarctica/',
 '/the-world-factbook/countries/antigua-and-barbuda/',
 '/the-world-factbook/countries/argentina/',
 '/the-world-factbook/countries/armenia/']

In [13]:
html.xpath('//span[@class="pagination__arrow-right"]') # this is the right-button

[<Element span at 0x7f871a36d810>]

In [15]:
driver.find_element("xpath", '//span[@class="pagination__arrow-right"]').click()

In [16]:
html=lx.fromstring(driver.page_source)
html.xpath('//h2/a/@href') # perfect. Now, build a pipeline 

['/the-world-factbook/countries/belize/',
 '/the-world-factbook/countries/benin/',
 '/the-world-factbook/countries/bermuda/',
 '/the-world-factbook/countries/bhutan/',
 '/the-world-factbook/countries/bolivia/',
 '/the-world-factbook/countries/bosnia-and-herzegovina/',
 '/the-world-factbook/countries/botswana/',
 '/the-world-factbook/countries/bouvet-island/',
 '/the-world-factbook/countries/brazil/',
 '/the-world-factbook/countries/british-indian-ocean-territory/',
 '/the-world-factbook/countries/british-virgin-islands/',
 '/the-world-factbook/countries/brunei/']

In [17]:
driver.quit()

In [18]:
import time
def get_countries():
    # set up driver
    driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()), 
    options = chrome_options)
    
    # get countries 
    countries = []
    driver.get(url)
    while True: 
        time.sleep(0.2)
        html=lx.fromstring(driver.page_source)
        newcountries=html.xpath('//h2/a/@href') 
        countries.extend(newcountries)
        try: # toggle the right-button if it exists... 
            driver.find_element("xpath", '//span[@class="pagination__arrow-right"]').click()
        except: #... else, break 
            break

    driver.quit()
    return countries

In [19]:
countries = get_countries()

In [20]:
len(countries) # the CIA WFB says there are 265 entities, but we only scraped 261. Well...

261

Now, lets get country-specific information! For whatever funny the url has to be changed. Find the correct URL via devtools. 

In [21]:
url = "https://www.cia.gov/the-world-factbook/page-data/countries/afghanistan/page-data.json"
r=requests.get(url)
r.json() # this was the correct URL

{'componentChunkName': 'component---src-templates-country-js',
 'path': '/countries/afghanistan/',
 'result': {'data': {'country': {'name': 'Afghanistan',
    'region': 'South Asia',
    'locations': ['place', 'summary'],
    'updated': 'October 29, 2024',
    'rank': True,
    'flag': {'childImageSharp': {'gatsbyImageData': {'layout': 'fullWidth',
       'images': {'fallback': {'src': '/the-world-factbook/static/2b3a06ae2c0b517dbfe6629f1f967718/2c686/AF-flag.jpg',
         'srcSet': '/the-world-factbook/static/2b3a06ae2c0b517dbfe6629f1f967718/2c686/AF-flag.jpg 452w',
         'sizes': '100vw'},
        'sources': [{'srcSet': '/the-world-factbook/static/2b3a06ae2c0b517dbfe6629f1f967718/66ca9/AF-flag.webp 452w',
          'type': 'image/webp',
          'sizes': '100vw'}]},
       'width': 1,
       'height': 0.6681415929203539}}},
    'map': {'childImageSharp': {'gatsbyImageData': {'layout': 'fullWidth',
       'images': {'fallback': {'src': '/the-world-factbook/static/694423d026c4bcbb

In [22]:
countries[0] # the format we have 

'/the-world-factbook/countries/afghanistan/'

In [23]:
countries[0].replace('/the-world-factbook/', '') # lets process the string

'countries/afghanistan/'

In [24]:
def get_info(country):
    time.sleep(0.2)
    c = country.replace('/the-world-factbook/', '')
    result = requests.get('https://www.cia.gov/the-world-factbook/page-data/' \
                          + c + 'page-data.json')
    result.raise_for_status()
    return result.json()

In [25]:
get_info(countries[0]) # bingo! 

{'componentChunkName': 'component---src-templates-country-js',
 'path': '/countries/afghanistan/',
 'result': {'data': {'country': {'name': 'Afghanistan',
    'region': 'South Asia',
    'locations': ['place', 'summary'],
    'updated': 'October 29, 2024',
    'rank': True,
    'flag': {'childImageSharp': {'gatsbyImageData': {'layout': 'fullWidth',
       'images': {'fallback': {'src': '/the-world-factbook/static/2b3a06ae2c0b517dbfe6629f1f967718/2c686/AF-flag.jpg',
         'srcSet': '/the-world-factbook/static/2b3a06ae2c0b517dbfe6629f1f967718/2c686/AF-flag.jpg 452w',
         'sizes': '100vw'},
        'sources': [{'srcSet': '/the-world-factbook/static/2b3a06ae2c0b517dbfe6629f1f967718/66ca9/AF-flag.webp 452w',
          'type': 'image/webp',
          'sizes': '100vw'}]},
       'width': 1,
       'height': 0.6681415929203539}}},
    'map': {'childImageSharp': {'gatsbyImageData': {'layout': 'fullWidth',
       'images': {'fallback': {'src': '/the-world-factbook/static/694423d026c4bcbb

In [26]:
countries_info = [get_info(country) for country in countries] # 261 requests, takes 2 min

Lets learn about ports!

In [33]:
countries_info[246]

{'componentChunkName': 'component---src-templates-country-js',
 'path': '/countries/united-states/',
 'result': {'data': {'country': {'name': 'United States',
    'region': 'North America',
    'locations': ['place', 'summary'],
    'updated': 'October 29, 2024',
    'rank': True,
    'flag': {'childImageSharp': {'gatsbyImageData': {'layout': 'fullWidth',
       'images': {'fallback': {'src': '/the-world-factbook/static/ee588cf04d974c70ba0b0169ed7e1050/24d46/US-flag.jpg',
         'srcSet': '/the-world-factbook/static/ee588cf04d974c70ba0b0169ed7e1050/24d46/US-flag.jpg 420w',
         'sizes': '100vw'},
        'sources': [{'srcSet': '/the-world-factbook/static/ee588cf04d974c70ba0b0169ed7e1050/2bfdb/US-flag.webp 420w',
          'type': 'image/webp',
          'sizes': '100vw'}]},
       'width': 1,
       'height': 0.5261904761904762}}},
    'map': {'childImageSharp': {'gatsbyImageData': {'layout': 'fullWidth',
       'images': {'fallback': {'src': '/the-world-factbook/static/256872aaa

In [32]:
countries_info[246]['result']['data']['fields']['nodes'] # this is a list of dicts...

[{'name': 'Age structure',
  'data': '<strong>0-14 years:</strong> 18.1% (male 31,618,532/female 30,254,223)<br><br><strong>15-64 years:</strong> 63.4% (male 108,553,822/female 108,182,491)<br><br><strong>65 years and over:</strong> 18.5% (2024 est.) (male 28,426,426/female 34,927,914)',
  'subfields': [{'label': '65 years and over', 'ranking': None},
   {'label': '15-64 years', 'ranking': None},
   {'label': '0-14 years', 'ranking': None}],
  'fieldLabel': [{'rank': False,
    'databaseId': '341',
    'toggleWithId': None,
    'toggleType': None,
    'subfieldLabels': [{'databaseId': None, 'rank': False},
     {'databaseId': None, 'rank': False},
     {'databaseId': None, 'rank': False}]}],
  'media': [{'type': 'image',
    'altText': '2023 population pyramid',
    'caption': "This is the population pyramid for United States. A population pyramid illustrates the age and sex structure of a country's population and may provide insights about political and social stability, as well as ec

Inspecting this JSON shows us that the info we are looking for is...

In [28]:
porttext = [i.get('data') for i in countries_info[246]['result']['data']['fields']['nodes'] \
             if i.get('name') == 'Ports'] # here!
porttext[0]

'<strong>total ports:</strong> 666 (2024)<br><br><strong>large:</strong> 21<br><br><strong>medium:</strong> 38<br><br><strong>small:</strong> 132<br><br><strong>very small:</strong> 475<br><br><strong>ports with oil terminals:</strong> 204<br><br><strong>key ports:</strong> Baltimore, Boston, Brooklyn, Buffalo, Chester, Cleveland, Detroit, Galveston, Houston, Los Angeles, Louisiana Offshore Oil Port (LOOP), Mobile, New Orleans, New York City, Norfolk, Oakland, Philadelphia, Portland, San Francisco, Seattle, Tri-City Port'

In [29]:
import re
keyporttext = re.findall('(?<=key ports:\<\/strong\>).*', porttext[0])
keyportlist = re.findall('(\w+[\s\w\-\(\)]*)', keyporttext[0])
keyportlist # this is what we want

['Baltimore',
 'Boston',
 'Brooklyn',
 'Buffalo',
 'Chester',
 'Cleveland',
 'Detroit',
 'Galveston',
 'Houston',
 'Los Angeles',
 'Louisiana Offshore Oil Port (LOOP)',
 'Mobile',
 'New Orleans',
 'New York City',
 'Norfolk',
 'Oakland',
 'Philadelphia',
 'Portland',
 'San Francisco',
 'Seattle',
 'Tri-City Port']

In [35]:
def ports(info):
    name = info['result']['data']['country']['name']
    try: 
        porttext = [i.get('data') for i in info['result']['data']['fields']['nodes'] \
                        if i.get('name') == 'Ports'] # here!
        keyporttext = re.findall('(?<=key ports:\<\/strong\>).*', porttext[0])
        keyportlist = re.findall('(\w+[\s\w\-\(\)]*)', keyporttext[0])
    except: 
        keyportlist = []
    return {name: keyportlist}

In [36]:
ports(countries_info[246])

{'United States': ['Baltimore',
  'Boston',
  'Brooklyn',
  'Buffalo',
  'Chester',
  'Cleveland',
  'Detroit',
  'Galveston',
  'Houston',
  'Los Angeles',
  'Louisiana Offshore Oil Port (LOOP)',
  'Mobile',
  'New Orleans',
  'New York City',
  'Norfolk',
  'Oakland',
  'Philadelphia',
  'Portland',
  'San Francisco',
  'Seattle',
  'Tri-City Port']}

In [37]:
keyports = [ports(info) for info in countries_info]

In [38]:
keyports[:10]

[{'Afghanistan': []},
 {'Akrotiri and Dhekelia': []},
 {'Albania': ['Durres', 'Shengjin', 'Vlores']},
 {'Algeria': ['Alger',
   'Annaba',
   'Arzew',
   'Arzew El Djedid',
   'Bejaia',
   'Mers El Kebir',
   'Oran',
   'Port Methanier',
   'Skikda']},
 {'American Samoa': ['Pago Pago Harbor']},
 {'Andorra': []},
 {'Angola': ['Cabinda',
   'Estrela Oil Field',
   'Lobito',
   'Luanda',
   'Malongo Oil Terminal',
   'Namibe',
   'Palanca Terminal',
   'Takula Terminal']},
 {'Anguilla': []},
 {'Antarctica': ['Admiralty Bay',
   'Andersen Harbor',
   'Ellefsen Harbor',
   'McMurdo Station',
   'Melchior Harbor',
   'Port Foster',
   'Port Lockroy',
   'Scotia Bay']},
 {'Antigua and Barbuda': ['St', 'John', 's']}]